In [1]:
"""Age
Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_TIME	 = Beta('B_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_AGE	 = Beta('B_AGE',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_AGE * AGE
SBB_SP = ASC_SBB * one + B_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST + B_AGE * AGE
SM_SP = ASC_SM * one + B_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MNL_SM_age"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.824
ASC_SM=	0.133
B_AGE=	0.199
B_CAR_COST=	-0.0094
B_FIRST=	0.31
B_GA=	0.487
B_HE=	-0.0055
B_SM_COST=	-0.0113
B_TIME=	-0.0113
B_TRAIN_COST=	-0.0297
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -0.823771  0.119176  -6.912196  4.772183e-12      0.138536   
ASC_SM        0.133269  0.111907   1.190894  2.336954e-01      0.126995   
B_AGE         0.199082  0.027766   7.169961  7.502887e-13      0.029930   
B_CAR_COST   -0.009398  0.000911 -10.312052  0.000000e+00      0.001174   
B_FIRST       0.310244  0.073461   4.223229  2.408269e-05      0.078032   
B_GA          0.487088  0.188045   2.590272  9.590003e-03      0.191640   
B_HE         -0.005500  0.001011  -5.439853  5.332445e-08      0.001015   
B_SM_COST    -0.011335  0.000598 -18.945473  0.000000e+00      0.000843   
B_TIME       -0.011250  0.000614 -18.332966  0.000000e+00      0.001220   
B_TRAIN_COST -0.029673  0.001200 -24.730352  0.000000e+00      0.001854   

              